# rag walkthrough\n这个 notebook 展示离线最小 RAG：读取报告、切分、检索、引用式回答。

In [ ]:
from rag.component import (\n    DEFAULT_QUERY,\n    chunk_report,\n    load_report_text,\n    retrieve_top_k,\n    run_demo,\n    synthesize_answer,\n)

In [ ]:
report_text = load_report_text()\nlen(report_text), report_text[:300]

In [ ]:
chunks = chunk_report(report_text)\nlen(chunks), chunks[0][:180]

In [ ]:
hits = retrieve_top_k(chunks, DEFAULT_QUERY, top_k=3)\nhits

In [ ]:
[(item['chunk_id'], round(item['score'], 4), item['text'][:120]) for item in hits]

In [ ]:
answer = synthesize_answer(DEFAULT_QUERY, hits)\nanswer

In [ ]:
result = run_demo()\nresult['trace'], result['final_answer']